# Eat Safe, Love

## Notebook Set Up

In [209]:
! mongoimport --type json -d uk_food -c establishments --drop --jsonArray ./../Resources/establishments.json

In [210]:
# Import dependencies
from pymongo import MongoClient
import json
import requests 
from pprint import pprint

In [211]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [212]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [213]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [214]:
# assign the collection to a variable
estab=db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [215]:
# Find the establishments with a hygiene score of 20
query ={'scores.Hygiene': 20}
field={'scores.Hygiene': 1, 'BusinessName': 1}
#query = [{'$group': {'_id': "$country", 'count': { '$sum': 1 }}}]
results=estab.find(query,field)

#for result in results:
   #print(result)
print(results[0])


{'_id': ObjectId('641d118b9a659a6a01be7942'), 'BusinessName': 'The Chase Rest Home', 'scores': {'Hygiene': 20}}


In [216]:
# Use count_documents to display the number of documents in the result
print("Number of documents in result:", estab.count_documents(query))

# Display the first document in the results using pprint
pprint(estab.find_one(query, field))


Number of documents in result: 41
{'BusinessName': 'The Chase Rest Home',
 '_id': ObjectId('641d118b9a659a6a01be7942'),
 'scores': {'Hygiene': 20}}


In [217]:

print(results[0])

{'_id': ObjectId('641d118b9a659a6a01be7942'), 'BusinessName': 'The Chase Rest Home', 'scores': {'Hygiene': 20}}


In [218]:
import pandas as pd
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)


# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(result_df))

# Display the first 10 rows of the DataFrame
result_df.head(10)

Rows in DataFrame:  41


,_id,BusinessName,scores
0,641d118b9a659a6a01be7942,The Chase Rest Home,{'Hygiene': 20}
1,641d118b9a659a6a01be7cc3,Brenalwood,{'Hygiene': 20}
2,641d118b9a659a6a01be7fe0,Melrose Hotel,{'Hygiene': 20}
3,641d118c9a659a6a01be81bd,Seaford Pizza,{'Hygiene': 20}
4,641d118c9a659a6a01be81cb,Golden Palace,{'Hygiene': 20}
5,641d118c9a659a6a01be8b6e,Ashby's Butchers,{'Hygiene': 20}
6,641d118c9a659a6a01be8d8f,South Sea Express Cuisine,{'Hygiene': 20}
7,641d118d9a659a6a01bea2c0,Golden Palace,{'Hygiene': 20}
8,641d118d9a659a6a01bea707,The Tulip Tree,{'Hygiene': 20}
9,641d118d9a659a6a01beaf1c,F & S,{'Hygiene': 20}


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [219]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.

query={'RatingValue': {'$gte': '4'}, 'LocalAuthorityName': {'$regex': 'London'}}

field={'LocalAuthorityName': 1,'RatingValue': 1}

sort=[('RatingValue', 1), ('LocalAuthorityName', 1)]

# Use count_documents to display the number of documents in the result
print("Number of documents in result:", estab.count_documents(query))

rating=list(estab.find(query, field).sort(sort))

# Display the first document in the results using pprint
pprint(rating)

Number of documents in result: 34
[{'LocalAuthorityName': 'City of London Corporation',
  'RatingValue': '4',
  '_id': ObjectId('641d118c9a659a6a01be936b')},
 {'LocalAuthorityName': 'City of London Corporation',
  'RatingValue': '4',
  '_id': ObjectId('641d118d9a659a6a01bea1dc')},
 {'LocalAuthorityName': 'City of London Corporation',
  'RatingValue': '4',
  '_id': ObjectId('641d118f9a659a6a01bedbcb')},
 {'LocalAuthorityName': 'City of London Corporation',
  'RatingValue': '4',
  '_id': ObjectId('641d11909a659a6a01bee29a')},
 {'LocalAuthorityName': 'City of London Corporation',
  'RatingValue': '5',
  '_id': ObjectId('641d118d9a659a6a01be968b')},
 {'LocalAuthorityName': 'City of London Corporation',
  'RatingValue': '5',
  '_id': ObjectId('641d118d9a659a6a01beafdf')},
 {'LocalAuthorityName': 'City of London Corporation',
  'RatingValue': '5',
  '_id': ObjectId('641d118d9a659a6a01beafe6')},
 {'LocalAuthorityName': 'City of London Corporation',
  'RatingValue': '5',
  '_id': ObjectId('641

In [220]:
# Convert the result to a Pandas DataFrame

result_df = pd.DataFrame(rating)

# Display the number of rows in the DataFrame

print("Rows in DataFrame: ", len(result_df))

# Display the first 10 rows of the DataFrame
result_df.head(10)

Rows in DataFrame:  34


,_id,RatingValue,LocalAuthorityName
0,641d118c9a659a6a01be936b,4,City of London Corporation
1,641d118d9a659a6a01bea1dc,4,City of London Corporation
2,641d118f9a659a6a01bedbcb,4,City of London Corporation
3,641d11909a659a6a01bee29a,4,City of London Corporation
4,641d118d9a659a6a01be968b,5,City of London Corporation
5,641d118d9a659a6a01beafdf,5,City of London Corporation
6,641d118d9a659a6a01beafe6,5,City of London Corporation
7,641d118e9a659a6a01beb6ed,5,City of London Corporation
8,641d118f9a659a6a01bed801,5,City of London Corporation
9,641d118f9a659a6a01bedbe5,5,City of London Corporation


### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [221]:
lat =estab.find_one({"BusinessName":"Penang Flavours"},{'geocode.latitude':1})['geocode']['latitude']
long =estab.find_one({"BusinessName":"Penang Flavours"},{'geocode.longitude':1})['geocode']['longitude']
lat=float(lat)
long=float(long)
ds = 0.01
print(f'latitube for "Penang Flavours" is {lat}, longitube for "Penang Flavours" is {long} ')
print(f'latitube for query is [{lat-ds}, {lat+ds}], longitube for query is [{long-ds}, {long+ds}]')

latitube for "Penang Flavours" is 51.490142, longitube for "Penang Flavours" is 0.08384 
latitube for query is [51.480142, 51.500142], longitube for query is [0.07384, 0.09383999999999999]


In [222]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

lat_b=str(lat-ds)
lat_t=str(lat+ds)
long_b=str(long-ds)
long_t=str(long+ds)
query={'geocode.latitude': 
        {'$gte':lat_b, '$lte':lat_t},'geocode.longitude': 
            {'$gte': long_b, '$lte': long_t}, 'RatingValue': '5'}

field={'BusinessName': 1, 'scores': 1, 'geocode': 1} 

limit=5
sort = [('scores.Hygiene', 1)]

In [223]:
hygiene=list(estab.find(query, field).sort(sort).limit(limit))
#hygiene=list(estab.find(query).sort(sort).limit(limit))

# Use count_documents to display the number of documents in the result
print("Number of documents in result:", estab.count_documents(query))

Number of documents in result: 87


In [224]:
# Convert result to Pandas DataFrame

hygiene_df = pd.DataFrame(hygiene)

print("Rows in DataFrame: ", len(hygiene_df))

hygiene_df.head()


Rows in DataFrame:  5


,_id,BusinessName,scores,geocode
0,641d118f9a659a6a01bed18e,Iceland,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...","{'longitude': '0.0924199968576431', 'latitude'..."
1,641d118f9a659a6a01bed19f,Howe and Co Fish and Chips - Van 17,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...","{'longitude': '0.0925370007753372', 'latitude'..."
2,641d118f9a659a6a01bed1bc,Volunteer,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...","{'longitude': '0.09208', 'latitude': '51.48734..."
3,641d118f9a659a6a01bed1d3,Plumstead Manor Nursery,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...","{'longitude': '0.0859939977526665', 'latitude'..."
4,641d118f9a659a6a01bed1d7,Atlantic Fish Bar,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...","{'longitude': '0.0912164', 'latitude': '51.486..."


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [225]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

match_query = {'$match': {'scores.Hygiene': 0}}

group_query = {'$group': {'_id': "$LocalAuthorityName", 
                          'count': { '$sum': 1 }}}

sort_values = {'$sort': { 'count': -1}}

# Put the pipeline together
pipeline = [match_query, group_query, sort_values]


In [226]:
results = list(estab.aggregate(pipeline))

# Print the number of documents in the result
print("Number of documents in result: ", len(results))


# Print the first 10 results
pprint(results[0:10])

Number of documents in result:  55
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [227]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(result_df))

# Display the first 10 rows of the DataFrame
result_df.head(10)

Rows in DataFrame:  55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
